In [2]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint

In [93]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_series_toregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['minus']=t_e['minus'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)

t_e=t_e.drop(['Unnamed: 0'],1)
t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)
t_e['ts-f1']=t_e['ts-f1'].abs()
t_e[t_e.watched_time_x > 20] = t_e[t_e.watched_time_x > 20].fillna(10.0)
t_e[t_e.watched_time_x > 10] = t_e[t_e.watched_time_x > 10].fillna(8.0)
t_e.watched_time_y=(t_e.watched_time_y/60)
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
from collections import Counter
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')
t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)
t_e=t_e.drop(['minus'],1 )

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])


t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])


t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)


t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)
t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)
t_e.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076152 entries, 0 to 1076151
Data columns (total 47 columns):
element_uid            1076152 non-null int32
user_uid               1076152 non-null int32
watched_time_x         1076152 non-null float64
rating                 245075 non-null float64
duration               1076152 non-null int64
feature_2              1076152 non-null float64
feature_3              1076152 non-null int16
feature_4              1076152 non-null float64
quantity               1076152 non-null int64
rating_mean            1053930 non-null float64
CM_S                   1076152 non-null int16
CM_P                   1076152 non-null int16
CM_R                   1076152 non-null int16
feature_5_65           1076152 non-null int16
feature_5_00           1076152 non-null int16
feature_5_68           1076152 non-null int16
feature_5_59           1076152 non-null int16
feature_5_-1           1076152 non-null int16
feature_5_44           1076152 non-null int16
smal

In [94]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
t_e=t_e.drop(['user_rating_mean'], 1)
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 

transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)


X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating


model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)

model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

X_train = transactions_exp[~transactions_exp.rating.isnull()]
#X_train.rating_mean=X_train.rating_mean.fillna(6.0)
X_train=X_train.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
#'user_rating_mean',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=4',
'Q_us_views=1'],1)

X_test=X_test.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
#'user_rating_mean',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=4',
'Q_us_views=1'],1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)


OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)

OLD=OLD.append(dobavit)
OLD.to_csv("SeriesRF4NEWbezsubmitaworst.csv")



In [95]:
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 

transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp=transactions_exp[transactions_exp.worst == 1]


X_test = transactions_exp
X_test=X_test.drop(['rating'],1)

submit1 = pd.DataFrame(model2.predict(X_test))
X_test=X_test.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
#'user_rating_mean',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=4',
'Q_us_views=1'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)
OLD1=pd.concat([X_test, submit1], 1)
OLD1['rating']=OLD1[0]
OLD1=OLD1.drop([0],1)

OLD=OLD.append(OLD1)
OLD.to_csv("SeriesRF4NEWdoobuch(ostatki).csv")
print('end all good')

end all good


In [61]:
OLD1['rating']=OLD1[0]
OLD1=OLD1.drop([0],1)




OLD=OLD.append(OLD1)
OLD.to_csv("SeriesRF4NEWdoobuch(ostatki).csv")
print('end all good')

end all good


In [53]:
OO.to_csv("SeriesRF4NEWdoobuch(ostatki).csv")

In [9]:
OO = pd.read_csv("SeriesRF4NEWdoobuch(ostatki).csv")

OO=OO.drop(['Unnamed: 0'], 1)
OO=OO.append(dobavit)


In [40]:
dobavit.to_csv("dobavit.csv")

In [3]:
OO = pd.read_csv("SeriesRF4NEWdoobuch(ostatki).csv")
main = pd.read_csv("SeriesRF4NEW.csv")
OLD = pd.read_csv("SeriesRF4NEWbezsubmitaworst.csv")
OLD=OLD.drop(['Unnamed: 0', 'worst'], 1)
OO=OO.drop(['Unnamed: 0', 'worst'], 1)
main=main.drop(['Unnamed: 0', 'worst'], 1)

In [4]:
OLD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893337 entries, 0 to 893336
Data columns (total 3 columns):
element_uid    893337 non-null int64
rating         893337 non-null float64
user_uid       893337 non-null int64
dtypes: float64(1), int64(2)
memory usage: 20.4 MB


In [5]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076152 entries, 0 to 1076151
Data columns (total 3 columns):
element_uid    1076152 non-null int64
rating         1076152 non-null float64
user_uid       1076152 non-null int64
dtypes: float64(1), int64(2)
memory usage: 24.6 MB


In [52]:
OO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076152 entries, 0 to 1076366
Data columns (total 3 columns):
element_uid    1076152 non-null int64
rating         1076152 non-null float64
user_uid       1076152 non-null int64
dtypes: float64(1), int64(2)
memory usage: 32.8 MB


In [9]:
OO[OO.duplicated() == True] 


,element_uid,rating,user_uid


In [104]:
main

,element_uid,rating,user_uid
0,5651,8.891720,490059
1,6274,9.316618,490059
2,5791,7.826436,490059
3,5899,8.412205,490059
4,3339,8.887437,490059
5,5826,8.888321,490059
6,5651,8.923259,278741
7,5791,7.720209,278741
8,5651,8.933725,469687
9,1328,8.925642,469687


In [113]:
end=pd.merge(OO, main, on=['element_uid', 'user_uid'])

In [114]:
end.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076367 entries, 0 to 1076366
Data columns (total 4 columns):
element_uid    1076367 non-null int64
rating_x       1076367 non-null float64
user_uid       1076367 non-null int64
rating_y       1076367 non-null float64
dtypes: float64(2), int64(2)
memory usage: 41.1 MB


In [116]:
end1=pd.concat([end['element_uid'], end['user_uid']],1)
end1.drop_duplicates(inplace=True)
end1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1076152 entries, 0 to 1076366
Data columns (total 2 columns):
element_uid    1076152 non-null int64
user_uid       1076152 non-null int64
dtypes: int64(2)
memory usage: 24.6 MB


In [44]:
end1=pd.concat([OO['element_uid'], OO['user_uid']],1)
end2=end1[end1.duplicated(keep= 'first') == True] 
t=end2.index

In [45]:
t

Int64Index([ 895351,  896922,  897129,  898520,  898551,  900664,  902275,
             902428,  902430,  902588,
            ...
            1069574, 1070228, 1071432, 1072660, 1073283, 1073315, 1074831,
            1075092, 1075308, 1075830],
           dtype='int64', length=215)

In [46]:
for i in t:
    print(OO.iloc[i].rating)

2.2305845964812465
2.183177986445852
2.85377262430189
5.2473114463080215
2.608502683341337
2.1664473332497622
3.063095716095308
2.230039208131525
2.0226356613700958
2.602986046904213
2.0280282066604025
2.2615221060950272
2.1325299548833234
2.971463554557676
2.463156559174246
2.085772805314558
2.6239320633923997
2.219977223774728
2.137145380126422
2.1648606846262504
4.528995130505945
2.0494544262112644
1.9989215772445632
6.535667898682228
6.389132129407807
5.23177564447467
2.1091911055703454
2.7808069086022935
2.2757891904971417
2.7607231969265933
2.126927178452869
2.2549486763964364
4.301815374598588
2.3041382566595834
4.180746700125754
2.2550454843751293
2.058717661673892
2.64324528703946
4.572397967035161
2.524559222486153
2.197083341239151
2.420665984736644
1.929356464114493
2.6435321876272964
2.32305955779676
3.830275521355763
2.3119960079726023
2.113366617953477
2.1066308354254337
2.393924709600033
3.2855397950091065
2.151206178873715
3.5042837725512905
2.4695806016934414
2.031633

In [51]:
OO.iloc[t]

,element_uid,rating,user_uid
895352,80,2.017679,132717
896924,2955,2.001878,346337
897132,8947,2.155354,56864
898524,4233,2.001286,273534
898556,2104,2.004084,401585
900670,7629,2.000995,394921
902282,2127,2.001024,528634
902437,1266,2.001294,161181
902439,8060,2.000946,401958
902598,1266,2.001293,181480


In [50]:
OO=OO.drop(index=t)

In [39]:
OO[OO.element_uid == 5175][  OO.user_uid == 380159]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,element_uid,rating,user_uid
653537,5175,10.000000,380159
896922,5175,2.183178,380159
